## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
print(data.head(5))
print(data.info())


   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жи

### Вывод

Данные не очень хорошие. Общий трудовой стаж отрицательный. Есть отрицательное количество детей. Много людей имеют по 20 детей, что странно. Много данных потеряно. 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:

data['days_employed_abs']=abs(data['days_employed'])
#print(data.head(5))
print(data['days_employed_abs'].isna().sum())
print(data['days_employed_abs'].median())
print(data['days_employed_abs'].min())
print(data['days_employed_abs'].mean())
print(data['days_employed_abs'].max())
print(data['dob_years'].max())
print('')
#print(data[data['days_employed_abs'].isnull()].head())

data['new']=data['dob_years'] - (data['days_employed_abs'])/365/24

data.loc[data['new']<14, ['days_employed_abs']] = data['days_employed_abs'].median()
#print(data[data['days_employed_abs']==data['days_employed_abs'].median()])

data['children_new']=abs(data['children'])
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXX')        
#print(data.loc[data['children_new']>15].head(15))

#data.loc[data['children_new']>15, ['children_new']] = -1
#print(data[data['children_new']==-1])
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXX')        
print(' ')
print(data['days_employed_abs'].mean())
print('')
data['days_employed_abs'] = data['days_employed_abs'].fillna(value=data['days_employed_abs'].mean())
#print(data[data['total_income'].isnull()].head())
print('')
print('')
print(data['days_employed_abs'].isna().sum())    
print(data['days_employed_abs'].value_counts().sum())
data['new_dob_years']=abs(data['dob_years'])
data.loc[data['new_dob_years']<18, ['new_dob_years']] = data['dob_years'].median()
#print(data[data['dob_years']<18])

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXX') 

2174
2194.220566878695
24.14163324048118
66914.72890682236
401755.40047533
75

XXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXX
 
49705.41765116333



0
21525
XXXXXXXXXXXXXXXXXXXXXXXXXXXX


In [3]:

print(data['total_income'].count())
print(data['total_income'].isnull().sum())

import numpy as np
print(data.pivot_table(index='income_type', values='total_income', aggfunc= np.nanmean))
#print(data[data['total_income'].isnull()])
profesii = data['income_type'].unique()

data_group_mean = data.groupby('income_type')['total_income'].mean()
 
def data_isnull(row):
    income_type = row['income_type']
    total_income = row['total_income']
    if pd.isnull(total_income) == False:
        return total_income
    if pd.isnull(total_income) == True:
        return data_group_mean[income_type]

data['total_income'] = data.apply(data_isnull, axis=1)


#data['total_income'] = data.groupby('income_type').transform(lambda x: x.fillna(x.mean()))

print(data['total_income'].count())
print(data['total_income'].isnull().sum())
print(data.head())


19351
2174
                  total_income
income_type                   
безработный      131339.751676
в декрете         53829.130729
госслужащий      170898.309923
компаньон        202417.461462
пенсионер        137127.465690
предприниматель  499163.144947
сотрудник        161380.260488
студент           98201.625314
21525
0
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0

### Вывод

Были заменины пропуски в соответствии с родом деятельности на среднее сначение

### Замена типа данных

In [4]:
data['days_employed_abs'] =data['days_employed_abs'].astype('int')
data['total_income'] =data['total_income'].astype('int')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 16 columns):
children             21525 non-null int64
days_employed        19351 non-null float64
dob_years            21525 non-null int64
education            21525 non-null object
education_id         21525 non-null int64
family_status        21525 non-null object
family_status_id     21525 non-null int64
gender               21525 non-null object
income_type          21525 non-null object
debt                 21525 non-null int64
total_income         21525 non-null int64
purpose              21525 non-null object
days_employed_abs    21525 non-null int64
new                  19351 non-null float64
children_new         21525 non-null int64
new_dob_years        21525 non-null float64
dtypes: float64(3), int64(8), object(5)
memory usage: 2.6+ MB
None


### Вывод

days_employed  и total_incomeна  заменены на int


### Обработка дубликатов

In [5]:
data['education_lowercase'] = data['education'].str.lower()
print(data['education_lowercase'].value_counts())
print(data['family_status_id'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education_lowercase, dtype: int64
0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


In [6]:
print(data.duplicated().sum())
#print(data[data.duplicated(keep = 'first')])
data = data.drop_duplicates().reset_index(drop = True)
print(data.duplicated().sum())

54
0


### Вывод

Заменены строковые значения и удалены дубликаты

### Лемматизация

In [7]:
from pymystem3 import Mystem
m = Mystem()
data_purpose=data[['debt','purpose']]
def purpose_group(purpose):
    lemmas_row=m.lemmatize(purpose)
    
    for word in lemmas_row:
        if word== 'недвижимость' :
            return 'операции с недвижимостью'
        if word== 'жилье' :
            return 'операции с недвижимостью'
        if word== 'строительство' :
            return 'операции с недвижимостью'
        if word== 'жилой' :
            return 'операции с недвижимостью'
        if word== 'коммерческий' :
            return 'операции с недвижимостью'
       
        if word== 'автомобиль' :
            return 'авто'
       
        if word== 'образование' :
            return 'образование'
       
        if word== 'свадьба' :
            return 'свадьба'
       
                   
data_purpose['purpose_group']=data_purpose['purpose'].apply(purpose_group)    
 
print(data_purpose['purpose_group'].value_counts())
          

операции с недвижимостью    10814
авто                         4308
образование                  4014
свадьба                      2335
Name: purpose_group, dtype: int64


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Вывод

Чаще всего люди берут кредиты на операции с недвижимостью. Потом берут кредиты на авто. Далее идет образование. И оказывается люди берут кредиты даже на свадьбы.


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
import numpy as np
def procent(a):   
    return np.mean(a)*100

#data_y_children = data['children'].sum()
data_y_children_y_debt = pd.pivot_table(data, values=['debt'], index= ['children_new'], aggfunc= {'debt': [procent, np.std]})
print(data_y_children_y_debt)





                   debt          
                procent       std
children_new                     
0              7.535266  0.263969
1              9.163921  0.288546
2              9.454191  0.292652
3              8.181818  0.274504
4              9.756098  0.300406
5              0.000000  0.000000
20            10.526316  0.308931


### Вывод

странное количество семей с 20 детьми данные надо проверить

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:

data_y_family_status_id_y_debt = pd.pivot_table(data, values=['debt'], index= ['family_status'], aggfunc= {'debt': [procent, np.std]})
print(data_y_family_status_id_y_debt)

                           debt          
                        procent       std
family_status                            
Не женат / не замужем  9.750890  0.296702
в разводе              7.112971  0.257149
вдовец / вдова         6.569343  0.247875
гражданский брак       9.320202  0.290750
женат / замужем        7.542126  0.264081


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:


print(data['total_income'].quantile(0.2))
print(data['total_income'].quantile(0.4))
print(data['total_income'].quantile(0.8))

data_02 = data[data['total_income']<data.groupby('debt')['total_income'].transform('quantile',0.2)]
data_y_total_income_y_debt = pd.pivot_table(data_02, values=['debt'], index= ['income_type'], aggfunc= {'debt': [procent, np.std, len]})
print(data_y_total_income_y_debt)
print('__________________________________')
data_04 = data[(data['total_income']>data.groupby('debt')['total_income'].transform('quantile',0.2)) & (data['total_income']<data.groupby('debt')['total_income'].transform('quantile',0.4))]
data_y_total_income_y_debt_04= pd.pivot_table(data_04, values=['debt'], index= ['income_type'], aggfunc= {'debt': [procent, np.std, len]})
print(data_y_total_income_y_debt_04)
print('__________________________________')
data_06 = data[(data['total_income']>data.groupby('debt')['total_income'].transform('quantile',0.4)) & (data['total_income']<data.groupby('debt')['total_income'].transform('quantile',0.6))]
data_y_total_income_y_debt_06= pd.pivot_table(data_06, values=['debt'], index= ['income_type'], aggfunc= {'debt': [procent, np.std, len]})
print(data_y_total_income_y_debt_06)
print('__________________________________')
data_08 = data[(data['total_income']>data.groupby('debt')['total_income'].transform('quantile',0.6)) & (data['total_income']<data.groupby('debt')['total_income'].transform('quantile',0.8))]
data_y_total_income_y_debt_08= pd.pivot_table(data_08, values=['debt'], index= ['income_type'], aggfunc= {'debt': [procent, np.std, len]})
print(data_y_total_income_y_debt_08)
print('__________________________________')
data_1 = data[data['total_income']>data.groupby('debt')['total_income'].transform('quantile',0.8)]
data_y_total_income_y_debt_1 = pd.pivot_table(data_1, values=['debt'], index= ['income_type'], aggfunc= {'debt': [procent, np.std, len]})
print(data_y_total_income_y_debt_1)

98554.0
135514.0
214565.0
               debt                      
                len     procent       std
income_type                              
безработный     1.0  100.000000       NaN
в декрете       1.0  100.000000       NaN
госслужащий   278.0    7.553957  0.264737
компаньон     557.0    8.078995  0.272757
пенсионер    1246.0    4.895666  0.215864
сотрудник    2210.0    9.909502  0.298857
студент         1.0    0.000000       NaN
__________________________________
               debt                    
                len   procent       std
income_type                            
госслужащий   282.0  6.028369  0.238435
компаньон     841.0  7.728894  0.267208
пенсионер     812.0  4.556650  0.208672
сотрудник    2358.0  9.669211  0.295601
__________________________________
               debt                    
                len   procent       std
income_type                            
госслужащий   185.0  5.405405  0.226738
компаньон     647.0  7.727975  0.267241
пенс

### Вывод

Сделала категории по уровню дохода и рассчитала проценты невозврата для каждому роду деятельности.

- Как разные цели кредита влияют на его возврат в срок?

In [12]:
data_y_purpose_group_id_y_debt = pd.pivot_table(data_purpose, values=['debt'], index= ['purpose_group'], aggfunc= {'debt': [procent, np.std]})
print(data_y_purpose_group_id_y_debt)



                              debt          
                           procent       std
purpose_group                               
авто                      9.354689  0.291231
образование               9.217738  0.289312
операции с недвижимостью  7.231367  0.259019
свадьба                   7.965739  0.270820
